# Advanced Robotics 
1. Ahmad Hamdan
2. Ghadeer Issa

In [18]:
import numpy as np
import roboticstoolbox as rtb
from spatialmath import *
from math import pi
import matplotlib.pyplot as plt
from matplotlib import cm
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

#%matplotlib notebook
%matplotlib Qt
#%matplotlib widgets

![title](img/banda.png)

## DH Parameters

In [19]:
robot = rtb.models.DH.Panda()                  # instantiate robot model
print(robot)

DHRobot: Panda (by Franka Emika), 7 joints (RRRRRRR), dynamics, geometry, modified DH parameters
┌────────┬────────┬─────┬───────┬─────────┬────────┐
│ aⱼ₋₁   │  ⍺ⱼ₋₁  │ θⱼ  │  dⱼ   │   q⁻    │   q⁺   │
├────────┼────────┼─────┼───────┼─────────┼────────┤
│    0.0 │   0.0° │  q1 │ 0.333 │ -166.0° │ 166.0° │
│    0.0 │ -90.0° │  q2 │   0.0 │ -101.0° │ 101.0° │
│    0.0 │  90.0° │  q3 │ 0.316 │ -166.0° │ 166.0° │
│ 0.0825 │  90.0° │  q4 │   0.0 │ -176.0° │  -4.0° │
│-0.0825 │ -90.0° │  q5 │ 0.384 │ -166.0° │ 166.0° │
│    0.0 │  90.0° │  q6 │   0.0 │   -1.0° │ 215.0° │
│  0.088 │  90.0° │  q7 │ 0.107 │ -166.0° │ 166.0° │
└────────┴────────┴─────┴───────┴─────────┴────────┘

┌─────┬──────┐
│tool │ None │
└─────┴──────┘

┌─────┬─────┬────────┬─────┬───────┬─────┬───────┬──────┐
│name │ q0  │ q1     │ q2  │ q3    │ q4  │ q5    │ q6   │
├─────┼─────┼────────┼─────┼───────┼─────┼───────┼──────┤
│  qr │  0° │ -17.2° │  0° │ -126° │  0° │  115° │  45° │
│  qz │  0° │  0°    │  0° │  0°   │  0° 

The first table shows the kinematic parameters, and from the column titles we can see clearly that this is expressed in terms of standard Denavit-Hartenberg parameters.  The first column shows that the joint variables qi are rotations since they are in the θ column.  Joint limits are also shown.  Joint flip (motion in the opposite sense) would be indicated by the joint variable being shown as for example like `-q3`.

The second table shows some named joint configurations.  For example `ur10.qr`

## Robot Model

In [20]:
robot = rtb.models.Panda()                  # instantiate robot model
print(robot)

ERobot: panda (by Franka Emika), 7 joints (RRRRRRR), 1 gripper, geometry, collision
┌─────┬──────────────┬───────┬─────────────┬────────────────────────────────────────────────┐
│link │     link     │ joint │   parent    │              ETS: parent to link               │
├─────┼──────────────┼───────┼─────────────┼────────────────────────────────────────────────┤
│   0 │ panda_link0  │       │ BASE        │                                                │
│   1 │ panda_link1  │     0 │ panda_link0 │ SE3(0, 0, 0.333) ⊕ Rz(q0)                      │
│   2 │ panda_link2  │     1 │ panda_link1 │ SE3(-90°, -0°, 0°) ⊕ Rz(q1)                    │
│   3 │ panda_link3  │     2 │ panda_link2 │ SE3(0, -0.316, 0; 90°, -0°, 0°) ⊕ Rz(q2)       │
│   4 │ panda_link4  │     3 │ panda_link3 │ SE3(0.0825, 0, 0; 90°, -0°, 0°) ⊕ Rz(q3)       │
│   5 │ panda_link5  │     4 │ panda_link4 │ SE3(-0.0825, 0.384, 0; -90°, -0°, 0°) ⊕ Rz(q4) │
│   6 │ panda_link6  │     5 │ panda_link5 │ SE3(90°, -0°, 0°) ⊕ Rz(q5

In [21]:
robot.plot(robot.qr)

Swift backend, t = 0.05, scene:
  panda

## Forward Kinematics

In [22]:
T = robot.fkine([-0.5, -0.5, -0.4, -0.4, -0.5, 0.6, 0.3])  # forward kinematics
print(T)

   0.5099   -0.8165    0.2709   -0.1506    
  -0.6557   -0.5727   -0.492    -0.1306    
   0.5568    0.07326  -0.8274    0.8901    
   0         0         0         1         



## Inverse
```ikine_LM ```is a generalised iterative numerical solution based on Levenberg-Marquadt minimization, and additional status results are also returned as part of a named tuple.

In [23]:
sol = robot.ikine_LM(T)                     # inverse kinematics
print(sol)

IKsolution(q=array([ -0.5582,  -0.4728,   -0.495,  -0.3804,  -0.3841,   0.6033,   0.2596]), success=True, reason=None, iterations=18, residual=7.920168289901014e-11)


## Plot 
Plotting the robot model and configuration using swift


In [24]:
robot.plot(sol.q, backend='swift')

Swift backend, t = 0.05, scene:
  panda

## Jacobian

Calculating Jacbian and simulating the movement between 2 points.

In [25]:
Je = robot.jacobe([0,0,0,0,0,0, 0])  
print(Je)

[[ 0.06223   0.3462  0.06223  -0.1228  0.06223   0.1488        0]
 [-0.06223   0.3462 -0.06223  -0.1228 -0.06223   0.1488        0]
 [       0    0.088        0  -0.0055        0   -0.088        0]
 [       0   0.7071        0  -0.7071        0  -0.7071        0]
 [       0  -0.7071        0   0.7071        0   0.7071        0]
 [      -1        0       -1        0       -1        0        1]]


In [26]:
#simulating trajectory between 2 configurations
traj = rtb.jtraj(robot.qr, sol.q, 100)
robot.plot(traj.q, backend='swift')

Swift backend, t = 4.99999999999999, scene:
  panda

## Dynamics

In [27]:
robot = rtb.models.DH.Panda()
#print(robot)
tau = robot.rne(robot.qr, np.zeros((7,)), np.zeros((7,)))
print(tau)

[       0    -10.6        0    14.59        0   0.7089        0]


In [28]:
robot.gravload(robot.qr)

array([       0,    -10.6,        0,    14.59,        0,   0.7089,        0])

In [29]:
robot.inertia(robot.qr)

array([[  0.7157, -0.009531,   0.7227, -0.001752,  0.01999, 0.001711, -0.004823],
       [-0.009531,    1.666, -0.008879,  -0.6112, 0.0008622, -0.03739,  0.00137],
       [  0.7227, -0.008879,   0.8504, -0.003926,    0.015, 0.001659, -0.004655],
       [-0.001752,  -0.6112, -0.003926,   0.7108, -0.0008622,  0.06189, -0.00137],
       [ 0.01999, 0.0008622,    0.015, -0.0008622,  0.02389, -0.0006332, 0.001711],
       [0.001711, -0.03739, 0.001659,  0.06189, -0.0006332,  0.03039, -0.00137],
       [-0.004823,  0.00137, -0.004655, -0.00137, 0.001711, -0.00137, 0.004815]])

The velocity terms are a bit harder to comprehend but they mean that rotation of one joint (and its link) can exert a torque on other joints. Consider that the should joint is rotating at 1 rad/sec, then the torque will be

# Calibration


In [46]:
from spatialmath import SE3
E1 = rtb.ET.tz(0.333)
E2 = rtb.ET.Rz()
E3 = rtb.ET.Ry()
E4 = rtb.ET.tz(0.316)
E5 = rtb.ET.Rz()
E6 = rtb.ET.tx(0.0825)
E7 = rtb.ET.Ry(flip=True)
E8 = rtb.ET.tx(-0.0825)
E9 = rtb.ET.tz(0.384)
E10 = rtb.ET.Rz()
E11 = rtb.ET.Ry(flip=True)
E12 = rtb.ET.tx(0.088)
E13 = rtb.ET.Rx(np.pi)
E14 = rtb.ET.tz(0.107)
E15 = rtb.ET.Rz()

# We can make an ETS representing a Panda by incorprating all 15 ETs into an ETS
panda = E1 * E2 * E3 * E4 * E5 * E6 * E7 * E8 * E9 * E10 * E11 * E12 * E13 * E14 * E15

# View the ETS
print(panda)
print()

q1=0
q2=0
q3=0
q4=0
q5=0
q6=0
q7=0

E1 = SE3.Tz(0.333)
E2 = SE3.Rz(q1)
E3 = SE3.Ry(q2)
E4 = SE3.Tz(0.316)
E5 = SE3.Rz(q3)
E6 = SE3.Tx(0.0825)
E7 = SE3.Ry(-q4)
E8 = SE3.Tx(-0.0825)
E9 = SE3.Tz(0.384)
E10 = SE3.Rz(q5)
E11 = SE3.Ry(-q6)
E12 = SE3.Tx(0.088)
E13 = SE3.Rx(np.pi)
E14 = SE3.Tz(0.210)
E15 = SE3.Rz((-np.pi/4) + q7) 

panda = E1 * E2 * E3 * E4 * E5 * E6 * E7 * E8 * E9 * E10 * E11 * E12 * E13 * E14 * E15
print(panda)

print(robot.fkine([0, 0, 0, -0, 0, -0, 0]))



tz(0.333) ⊕ Rz(q0) ⊕ Ry(q1) ⊕ tz(0.316) ⊕ Rz(q2) ⊕ tx(0.0825) ⊕ Ry(-q3) ⊕ tx(-0.0825) ⊕ tz(0.384) ⊕ Rz(q4) ⊕ Ry(-q5) ⊕ tx(0.088) ⊕ Rx(180°) ⊕ tz(0.107) ⊕ Rz(q6)

   0.7071    0.7071    0         0.088     
   0.7071   -0.7071    0         0         
   0         0        -1         0.823     
   0         0         0         1         

   0.7071    0.7071    0         0.088     
   0.7071   -0.7071    0         0         
   0         0        -1         0.823     
   0         0         0         1         



### Utils
a collection of tranformation matrix

In [49]:
cos=np.cos
sin=np.sin
pi=np.pi


def Rxd(phi):
    rx = np.array([[ 0,         0,         0, 0],
                   [0, -sin(phi),  cos(phi), 0],
                   [0, -cos(phi), -sin(phi), 0],
                   [0,         0,         0, 0]])
    return SE3(rx, check=False)

def Ryd(phi):
    ry = np.array([[ -sin(phi), 0,  cos(phi), 0],
                   [0,         0,         0, 0],
                   [-cos(phi), 0, -sin(phi), 0],
                   [0,         0,         0, 0]])
    return SE3(ry, check=False)

def Rzd(phi):
    rz = np.array([[ -sin(phi),  cos(phi), 0, 0],
                   [-cos(phi), -sin(phi), 0, 0],
                   [0,         0,         0, 0],
                   [0,         0,         0, 0]])
    return SE3(rz, check=False)
Txd = np.array([[0,   0,  0,  1],
                   [0,   0,  0,  0],
                   [0,   0,  0,  0],
                   [0,   0,  0,  0]])
Tyd=np.array([[0,   0,  0,  0],
                   [0,   0,  0,  1],
                   [0,   0,  0,  0],
                   [0,   0,  0,  0]])
Tzd= np.array([[0,   0,  0,  0],
                   [0,   0,  0,  0],
                   [0,   0,  0,  1],
                   [0,   0,  0,  0]])

Txd=SE3(Txd, check=False)
Tyd=SE3(Tyd, check=False)
Tzd=SE3(Tzd, check=False)


## Irreducable model derivation:

$T=[T_xT_yT_zR_xR_yR_z]_b.R_z(q_1+\Delta q_1).[T_xT_yR_xR_y]_{L_1}.R_y(q_2+\Delta q_2).[T_xT_zR_xR_z]_{L_2}.
R_y(q_3+\Delta q_3).[T_xT_zR_xR_z]_{L_3}.R_z(q_4+\Delta q_4).[T_xT_yR_xR_y]_{L_4}.R_y(q_5+\Delta q_5).[T_xT_zR_xR_z]_{L_5}.
R_z(q_6+\Delta q_6).[T_xT_yR_xR_y]_{L_6}.[T_xT_yT_zR_xR_yR_z]_t$



$T=[T_xT_yT_zR_xR_yR_z]_b.R_z(q_1+\cancel{\Delta q_1}).[T_xT_yR_x\cancel{R_y}]_{L_1}.R_y(q_2+\Delta q_2).[T_x\cancel{T_z}R_xR_z]_{L_2}.
R_y(q_3+\Delta q_3).[T_xT_zR_x\cancel{R_z}]_{L_3}.R_z(q_4+\Delta q_4).[T_xT_yR_x\cancel{R_y}]_{L_4}.R_y(q_5+\Delta q_5).[T_x\cancel{T_z}R_x\cancel{R_z}]_{L_5}.
R_z(q_6+\cancel{\Delta q_6}).[\cancel{T_x}\cancel{T_y}\cancel{R_x}\cancel{R_y}]_{L_6}.[T_xT_yT_z\cancel{R_x}\cancel{R_y}\cancel{R_z}]_t$


$T_{robot}=R_z(q_1).[T_xT_yR_x]_{L_1}.R_y(q_2+\Delta q_2).[T_xR_xR_z]_{L_2}.R_y(q_3+\Delta q_3).[T_xT_zR_x]_{L_3}.
R_z(q_4+\Delta q_4).[T_xT_yR_x]_{L_4}.R_y(q_5+\Delta q_5).[T_xR_x]_{L_5}.R_z(q_6)$



$\mathbf{\pi}=\{ p_{x1} \quad p_{y1} \quad \varphi_{x1} \quad \Delta q_2 \quad p_{x2} \quad \varphi_{x2} \quad \varphi_{z2}
\quad \Delta q_3 \quad p_{x3} \quad p_{z3} \quad \varphi_{x3} \quad \Delta q_4 \quad p_{x4} \quad p_{y4} \quad \varphi_{x4} \quad
\Delta q_5 \quad p_{x5} \quad \varphi_{x6}
\}$


$\mathbf{\pi}_0=\{ 0 \quad -d_2 \quad 0 \quad 0 
\quad d_3 \quad 0 \quad 0 \quad 0
\quad d_4 \quad d_6 \quad 0 \quad 0 
 \quad 0 \quad d_5 \quad 0 \quad 0 
 \quad 0 \quad 0
\}$